# Author-Topic Modeling
### Source: https://nbviewer.jupyter.org/github/rare-technologies/gensim/blob/develop/docs/notebooks/atmodel_tutorial.ipynb

## 1. Processing Text, Vectorize Author List

In [1]:
import nltk
import json
content = []

content = []
with open('total.json') as json_data:
    content = json.load(json_data)

#for line in open('JACS.json', 'r'):
#    content.append(json.loads(line))
    
print(len(content))


5676


In [2]:
# Get all author names and their corresponding document IDs.
author2doc = dict()

i = 0
for entry in content:
    sender = entry['Sender'].replace('\n',' ')
    if not author2doc.get(sender):
        # This is a new author.
        #author2doc[sender] = []
        author2doc[sender] = [i]
    # Add document IDs to author.
    else:
        author2doc[sender].append(i)
    i = i + 1
    
i = 0    
for entry in content:
    receiver = entry['Receiver'].replace('\n',' ')
    if not author2doc.get(receiver):
        # This is a new author.
        author2doc[receiver] = []
        author2doc[receiver] = [i]
    # Add document IDs to author.
    else:
        author2doc[receiver].append(i)
    i = i + 1
    
    
#print(author2doc)


In [11]:
import spacy
nlp = spacy.load('en')

In [12]:
import re
abstract = []
for entry in content:
    title = entry['Title'].replace('\n',' ')
    title = title.replace('/u',' ')
    #sender = entry['Sender'].replace('\n',' ')
    #receiver = entry['Receiver'].replace('\n',' ')
    abst = entry['Content'].replace('\n',' ')
    abst = abst.replace('/u',' ')
    entry_str = title+' '+abst
    entry_str = re.sub(r'\b\w{1,3}\b', '',entry_str)
    abstract.append(entry_str)
    
#print(abstract[10])

In [13]:
from nltk.corpus import stopwords
d = {}
stopword = stopwords.words('english')

In [14]:
%%time
processed_docs = []    
for doc in nlp.pipe(abstract, n_threads=4, batch_size=100):
    # Process document using Spacy NLP pipeline.
    
    ents = doc.ents  # Named entities.

    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove common words from a stopword list.
    doc = [token for token in doc if token not in stopword]

    # Add named entities, but only if they are a compound of more than word.
    doc.extend([str(entity) for entity in ents if len(entity) > 1])
    
    processed_docs.append(doc)

CPU times: user 5min 43s, sys: 1min 22s, total: 7min 5s
Wall time: 5min 19s


In [15]:
abstract_all = processed_docs
del processed_docs

In [20]:
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(abstract_all, min_count=20)
for idx in range(len(abstract_all)):
    for token in bigram[abstract_all[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            abstract_all[idx].append(token)

/Users/Maggie/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [24]:
from gensim.corpora import Dictionary
dictionary = Dictionary(abstract_all)

# Remove rare and common tokens.
# Filter out words that occur too frequently or too rarely.
max_freq = 0.2
min_wordcount = 80
dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)

_ = dictionary[0]  # This sort of "initializes" dictionary.id2token.

In [25]:
corpus = [dictionary.doc2bow(doc) for doc in abstract_all]

In [26]:
print('Number of authors: %d' % len(author2doc))
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of authors: 10345
Number of unique tokens: 1156
Number of documents: 5676


In [248]:
#print(len(corpus))
#print(len(dictionary.id2token))

## 2. Select Best Performing Model with highest coherence

In [250]:
## What if I first make the corpus a tfidf corpus?

In [47]:
import gensim
from gensim import corpora,models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
#print(corpus[10])
#print(corpus_tfidf[10])
#print(type(corpus_tfidf))
#print(l[30]

dictionary = corpora.Dictionary(abstract_all)

#doc_term_matrix = [dictionary.doc2bow(doc) for doc in abstract_all]

#tfidf = models.TfidfModel(doc_term_matrix)
#corpus_tfidf = tfidf[doc_term_matrix]

l = list()
for t in corpus_tfidf:
    l.append(t)
    
print(l[10])


#lda_model_tfidf = models.LdaModel(corpus_tfidf, num_topics=5,id2word = dictionary)
#for idx, topic in lda_model_tfidf.print_topics(-1):
#    print('Topic: {} \nWords: {}'.format(idx+1, topic))

[(15, 0.054385169892930255), (39, 0.053780755777230696), (66, 0.06104200233455314), (70, 0.5162322190104439), (79, 0.12354232671973162), (80, 0.19824536448602734), (87, 0.08098196791570465), (112, 0.04920023169440311), (117, 0.20617140961712968), (145, 0.10775635159084852), (148, 0.1057658358717933), (156, 0.07548354088618732), (157, 0.09261727132159699), (159, 0.056549131420609584), (166, 0.06451582600974501), (173, 0.07127690361965881), (181, 0.14868962741285474), (185, 0.076532969738671), (188, 0.20427755365854164), (189, 0.0723040945047453), (199, 0.18128106759033505), (207, 0.09000927702053883), (214, 0.07144570199059921), (240, 0.0894930936809928), (287, 0.06447097148013425), (327, 0.09074703383177096), (328, 0.11585578306929534), (329, 0.30688367181371473), (330, 0.225229220597977), (331, 0.12590437687828407), (332, 0.10851387177713308), (333, 0.41893123018914513), (334, 0.11948566708307701), (335, 0.10559120006143236), (336, 0.08381573538528338), (337, 0.12875817689683616), (33

In [49]:
from gensim.models import AuthorTopicModel
%time model = AuthorTopicModel(corpus=l, num_topics=5, id2word=dictionary, \
                author2doc=author2doc, chunksize=2000, passes=1, eval_every=0, \
                iterations=1, random_state=1)
model.show_topics(num_topics=5)

CPU times: user 8.17 s, sys: 168 ms, total: 8.34 s
Wall time: 9.75 s


[(0,
  '0.000*"smell" + 0.000*"upper" + 0.000*"haemolysis" + 0.000*"elcid" + 0.000*"equipoise" + 0.000*"stigmatisation" + 0.000*"    week" + 0.000*" weeks\'" + 0.000*"adjudicator" + 0.000*"between 2011  2015"'),
 (1,
  '0.000*"smell" + 0.000*"upper" + 0.000*"haemolysis" + 0.000*"elcid" + 0.000*"equipoise" + 0.000*"stigmatisation" + 0.000*"    week" + 0.000*" weeks\'" + 0.000*"adjudicator" + 0.000*"between 2011  2015"'),
 (2,
  '0.000*"smell" + 0.000*"upper" + 0.000*"haemolysis" + 0.000*"elcid" + 0.000*"equipoise" + 0.000*"stigmatisation" + 0.000*"    week" + 0.000*" weeks\'" + 0.000*"adjudicator" + 0.000*"between 2011  2015"'),
 (3,
  '0.000*"smell" + 0.000*"upper" + 0.000*"haemolysis" + 0.000*"elcid" + 0.000*"equipoise" + 0.000*"stigmatisation" + 0.000*"    week" + 0.000*" weeks\'" + 0.000*"adjudicator" + 0.000*"between 2011  2015"'),
 (4,
  '0.000*"upper" + 0.000*"smell" + 0.000*"haemolysis" + 0.000*"elcid" + 0.000*"equipoise" + 0.000*"stigmatisation" + 0.000*"    week" + 0.000*" wee

In [28]:
%%time
from gensim.models import AuthorTopicModel
model_list = []
for i in range(5):
    model = AuthorTopicModel(corpus=corpus, num_topics=20, id2word=dictionary.id2token, \
                    author2doc=author2doc, chunksize=2000, passes=100, gamma_threshold=1e-10, \
                    eval_every=0, iterations=1, random_state=i)
    top_topics = model.top_topics(corpus)
    tc = sum([t[1] for t in top_topics])
    model_list.append((model, tc))

CPU times: user 12min 7s, sys: 1min 10s, total: 13min 18s
Wall time: 13min 58s


In [51]:
print(l[10])
print(corpus[10])

[(15, 0.054385169892930255), (39, 0.053780755777230696), (66, 0.06104200233455314), (70, 0.5162322190104439), (79, 0.12354232671973162), (80, 0.19824536448602734), (87, 0.08098196791570465), (112, 0.04920023169440311), (117, 0.20617140961712968), (145, 0.10775635159084852), (148, 0.1057658358717933), (156, 0.07548354088618732), (157, 0.09261727132159699), (159, 0.056549131420609584), (166, 0.06451582600974501), (173, 0.07127690361965881), (181, 0.14868962741285474), (185, 0.076532969738671), (188, 0.20427755365854164), (189, 0.0723040945047453), (199, 0.18128106759033505), (207, 0.09000927702053883), (214, 0.07144570199059921), (240, 0.0894930936809928), (287, 0.06447097148013425), (327, 0.09074703383177096), (328, 0.11585578306929534), (329, 0.30688367181371473), (330, 0.225229220597977), (331, 0.12590437687828407), (332, 0.10851387177713308), (333, 0.41893123018914513), (334, 0.11948566708307701), (335, 0.10559120006143236), (336, 0.08381573538528338), (337, 0.12875817689683616), (33

In [29]:
model, tc = max(model_list, key=lambda x: x[1])
print('Topic coherence: %.3e' %tc)

Topic coherence: -4.551e+01


In [30]:
model.save('/tmp/model.atmodel')

In [31]:
model = AuthorTopicModel.load('/tmp/model.atmodel')

In [361]:
#model.show_topics(num_topics=20)

In [35]:
topics = []
i = 1
for topic in model.show_topics(num_topics=20):
    words = []
    for word, prob in model.show_topic(topic[0]):
        words.append(word)
    print('Topic '+str(i)+': ')
    print(words[2]+' '+words[1]+' '+words[0])
    print(*words)
    print()
    i += 1
    topics.append(words[2]+' '+words[1]+' '+words[0])

Topic 1: 
value level model
model level value parameter predict post sample correlation change serum

Topic 2: 
temperature state surface
surface state temperature energy structure electron material free density dynamic

Topic 3: 
molecule compound structure
structure compound molecule cluster synthesis approach yield organic small product

Topic 4: 
cognitive adult child
child adult cognitive disorder exposure community stimulation drug social healthy

Topic 5: 
bond metal reaction
reaction metal bond complex catalyst ligand intermediate formation catalytic synthesis

Topic 6: 
image information model
model information image individual learning propose social learn feature scale

Topic 7: 
positive testing test
test testing positive negative infection sample injury sensitivity diagnostic laboratory

Topic 8: 
condition self experiment
experiment self condition object light stimulus natural human perception motion

Topic 9: 
approach cost case
case cost approach research clinical evide

In [36]:
from pprint import pprint

def show_author(name):
    print('\n%s' % name)
    print('Docs:', model.author2doc[name])
    print('Topics:')
    pprint([(topics[topic[0]],topic[1]) for topic in model[name]])

In [37]:
show_author('Jonathan L. Sessler')


Jonathan L. Sessler
Docs: [4005, 4139, 4308, 4706, 4809, 4811]
Topics:
[('temperature state surface', 0.810837091635193),
 ('bond metal reaction', 0.1334760484694536),
 ('condition self experiment', 0.048146385425494896)]


## 3. Plotting the Authors

We take all the author-topic distributions (stored in model.state.gamma) and embed them in a 2D space. To do this, we reduce the dimensionality of this data using t-SNE.

t-SNE is a method that attempts to reduce the dimensionality of a dataset, while maintaining the distances between the points. That means that if two authors are close together in the plot below, then their topic distributions are similar.

In the cell below, we transform the author-topic representation into the t-SNE space. You can increase the smallest_author value if you do not want to view all the authors with few documents.



In [38]:
%%time
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
smallest_author = 0  # Ignore authors with documents less than this.
authors = [model.author2id[a] for a in model.author2id.keys() if len(model.author2doc[a]) >= smallest_author]
_ = tsne.fit_transform(model.state.gamma[authors, :])  # Result stored in tsne.embedding_

CPU times: user 7min 21s, sys: 31.9 s, total: 7min 53s
Wall time: 5min 46s


In [40]:
# Tell Bokeh to display plots inside the notebook.
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [41]:
from bokeh.models import HoverTool
from bokeh.plotting import figure, show, ColumnDataSource

x = tsne.embedding_[:, 0]
y = tsne.embedding_[:, 1]

author_names = [model.id2author[a] for a in authors]

# Radius of each point corresponds to the number of documents attributed to that author.
scale = 0.4
author_sizes = [len(model.author2doc[a]) for a in author_names]
radii = [size * scale for size in author_sizes]

source = ColumnDataSource(
        data=dict(
            x=x,
            y=y,
            author_names=author_names,
            author_sizes=author_sizes,
            radii=radii,
        )
    )

# Add author names and sizes to mouse-over info.
hover = HoverTool(
        tooltips=[
        ("author", "@author_names"),
        ("size", "@author_sizes"),
        ]
    )

p = figure(tools=[hover, 'crosshair,pan,wheel_zoom,box_zoom,reset,save,lasso_select'])
p.scatter('x', 'y', radius='radii', source=source, fill_alpha=0.6, line_color=None)
show(p)

## 4. Similarity Queries

In [170]:
from gensim.similarities import MatrixSimilarity

# Generate a similarity object for the transformed corpus.
index = MatrixSimilarity(model[list(model.id2author.values())])

# Get similarities to some author.
author_name = 'Yadong Li'
sims = index[model[author_name]]

/Users/Maggie/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [42]:
# Make a function that returns similarities based on the Hellinger distance.

from gensim import matutils
import pandas as pd

# Make a list of all the author-topic distributions.
author_vecs = [model.get_author_topics(author) for author in model.id2author.values()]

def similarity(vec1, vec2):
    '''Get similarity between two vectors'''
    dist = matutils.hellinger(matutils.sparse2full(vec1, model.num_topics), \
                              matutils.sparse2full(vec2, model.num_topics))
    sim = 1.0 / (1.0 + dist)
    return sim

def get_sims(vec):
    '''Get similarity of vector to all authors.'''
    sims = [similarity(vec, vec2) for vec2 in author_vecs]
    return sims

def get_table(name, top_n=10, smallest_author=1):
    '''
    Get table with similarities, author names, and author sizes.
    Return `top_n` authors as a dataframe.
    
    '''
    
    # Get similarities.
    sims = get_sims(model.get_author_topics(name))

    # Arrange author names, similarities, and author sizes in a list of tuples.
    table = []
    for elem in enumerate(sims):
        author_name = model.id2author[elem[0]]
        sim = elem[1]
        author_size = len(model.author2doc[author_name])
        if author_size >= smallest_author:
            table.append((author_name, sim, author_size))
            
    # Make dataframe and retrieve top authors.
    df = pd.DataFrame(table, columns=['Author', 'Score', 'Size'])
    df = df.sort_values('Score', ascending=False)[:top_n]
    
    return df

In [43]:
get_table('Jonathan L. Sessler')

,Author,Score,Size
4273,Jonathan L. Sessler,1.000000,6
2888,Greg A. Mutch,0.974725,1
7101,Prashant V. Kamat,0.967393,1
2332,Eric A. Marro,0.893052,1
2283,Emil Karshalev,0.890042,1
4309,Jordi Martínez-Esaín,0.889173,1
8767,Thomas Bein,0.871466,1
1300,Chad A. Mirkin,0.866030,5
6953,Peng Teng,0.865584,1
3646,J. Paul Attfield,0.865475,1


In [44]:
get_table('Jonathan L. Sessler', smallest_author=3)

,Author,Score,Size
46,Jonathan L. Sessler,1.000000,6
13,Chad A. Mirkin,0.866030,5
70,Niels H. Damrauer,0.847869,3
122,Yi Xie,0.844644,6
9,Ben Zhong Tang,0.841964,4
128,Yuan Pan,0.815481,3
68,Mircea Dincă,0.792263,4
21,Feng Wang,0.790912,4
111,William A. Goddard,0.771804,4
113,Xianran Xing,0.764265,3


In [45]:
author_dict = {}
for a in author2doc:
    topic = [(topics[t[0]]) for t in model[a]]
    author_dict[a] = topic

In [46]:
pd.DataFrame.from_dict(author_dict, orient='index')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Thomas R Vetter,positive testing test,approach cost case,medical care health,male symptom group,meta_analysis meta risk,face response network,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Markus Müller-Trapet,molecule compound structure,image information model,approach cost case,visual task experiment,face response network,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Bashir Mohammed Abubakar,molecule compound structure,positive testing test,plant expression gene,enzyme activity protein,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Ye Han,value level model,molecule compound structure,image information model,tumor cancer cell,therapy trial treatment,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Chang Wang,image information model,blood rate system,disease finding case,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Matthias Steinfath,image information model,positive testing test,approach cost case,visual task experiment,meta_analysis meta risk,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Irene Ma,value level model,cognitive adult child,positive testing test,medical care health,therapy trial treatment,meta_analysis meta risk,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Mustapha Raïssouli,value level model,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Sebastián Cea-Echenique,bond metal reaction,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Peter Gärdenfors,temperature state surface,molecule compound structure,image information model,approach cost case,enzyme activity protein,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
